# 参数管理

在选择了架构并设置了超参数后，我们就进入了训练阶段。
此时，我们的目标是找到使损失函数最小化的模型参数值。
经过训练后，我们将需要使用这些参数来做出未来的预测。
此外，有时我们希望提取参数，以便在其他环境中复用它们，
将模型保存下来，以便它可以在其他软件中执行，
或者为了获得科学的理解而进行检查。

之前的介绍中，我们只依靠深度学习框架来完成训练的工作，
而忽略了操作参数的具体细节。
本节，我们将介绍以下内容：

* 访问参数，用于调试、诊断和可视化；
* 参数初始化；
* 在不同模型组件间共享参数。

(**我们首先看一下具有单隐藏层的多层感知机。**)

In [1]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))

X, net(X)

(tensor([[0.9473, 0.1408, 0.5314, 0.4237],
         [0.1545, 0.3516, 0.3701, 0.4959]]),
 tensor([[0.4900],
         [0.2609]], grad_fn=<AddmmBackward0>))

## [**参数访问**]

我们从已有模型中访问参数。
当通过`Sequential`类定义模型时，
我们可以通过索引来访问模型的任意层。
这就像模型是一个列表一样，每层的参数都在其属性中。
如下所示，我们可以检查第二个全连接层的参数。

In [2]:
print(net[0].state_dict())

OrderedDict([('weight', tensor([[-0.1499,  0.3554,  0.3243, -0.1847],
        [ 0.0455,  0.3521, -0.2007,  0.0680],
        [ 0.4860, -0.2394,  0.4372, -0.0590],
        [ 0.3551,  0.2468, -0.2327,  0.1194],
        [-0.3683, -0.0791,  0.4001,  0.2472],
        [-0.4399,  0.2609, -0.0626, -0.1922],
        [ 0.4200, -0.4823, -0.1344,  0.1079],
        [ 0.0541,  0.1088, -0.4551, -0.4700]])), ('bias', tensor([ 0.0601, -0.2213,  0.3355,  0.2891,  0.3082,  0.2305,  0.1661,  0.3045]))])


输出的结果告诉我们一些重要的事情：
首先，这个全连接层包含两个参数，分别是该层的权重和偏置。
两者都存储为单精度浮点数（float32）。
注意，参数名称允许唯一标识每个参数，即使在包含数百个层的网络中也是如此。

### [**目标函数**]

注意，每个参数都表示为参数类的一个实例。
要对参数执行任何操作，首先我们需要访问底层的数值。
有几种方法可以做到这一点。有些比较简单，而另一些则比较通用。
下面的代码从第二个全连接层（即第三个神经网络层）提取偏置，
提取后返回的是一个参数类实例，并进一步访问该参数的值。

In [3]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.1968], requires_grad=True)
tensor([0.1968])


参数是复合的对象，包含值、梯度和额外信息。
这就是我们需要显式参数值的原因。
除了值之外，我们还可以访问每个参数的梯度。
在上面这个网络中，由于我们还没有调用反向传播，所以参数的梯度处于初始状态。

In [4]:
net[2].weight.grad == None

True

### [**一次性访问所有参数**]

当我们需要对所有参数执行操作时，逐个访问它们可能会很麻烦。
当我们处理更复杂的块（例如，嵌套块）时，情况可能会变得特别复杂，
因为我们需要递归整个树来提取每个子块的参数。
下面，我们将通过演示来比较访问第一个全连接层的参数和访问所有层。

In [5]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


[为什么0.weight的size是[8, 4], 2.weight的size是[1, 8]，而且没有1.weight](https://kimi.moonshot.cn/share/cql4u0mc2kuhg8966hg0)

这为我们提供了另一种访问网络参数的方式，如下所示。

In [6]:
net.state_dict()['2.bias'].data

tensor([0.1968])

### [**从嵌套块收集参数**]

让我们看看，如果我们将多个块相互嵌套，参数命名约定是如何工作的。
我们首先定义一个生成块的函数（可以说是“块工厂”），然后将这些块组合到更大的块中。

In [7]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), 
                        nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.4482],
        [0.4482]], grad_fn=<AddmmBackward0>)

[**设计了网络后，我们看看它是如何工作的。**]

In [8]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


因为层是分层嵌套的，所以我们也可以像通过嵌套列表索引一样访问它们。
下面，我们访问第一个主要的块中、第二个子块的第一层的偏置项。

In [9]:
rgnet[0][1][0].bias.data

tensor([-0.1229, -0.1921,  0.2069, -0.0311, -0.4421,  0.4749, -0.3397,  0.2542])

## 参数初始化

知道了如何访问参数后，现在我们看看如何正确地初始化参数。
我们在[数值稳定性](https://github.com/lixinjie97/Deep_learning_tutorial/blob/main/03.multilayer-perceptrons/09.numerical-stability-and-init.ipynb)中讨论了良好初始化的必要性。
深度学习框架提供默认随机初始化，
也允许我们创建自定义初始化方法，
满足我们通过其他规则实现初始化权重。

默认情况下，PyTorch会根据一个范围均匀地初始化权重和偏置矩阵，
这个范围是根据输入和输出维度计算出的。
PyTorch的`nn.init`模块提供了多种预置初始化方法。

### [**内置初始化**]

让我们首先调用内置的初始化器。
下面的代码将所有权重参数初始化为标准差为0.01的高斯随机变量，
且将偏置参数设置为0。

In [10]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0047, -0.0136, -0.0024, -0.0033]), tensor(0.))

我们还可以将所有参数初始化为给定的常数，比如初始化为1。

In [11]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

我们还可以[**对某些块应用不同的初始化方法**]。
例如，下面我们使用Xavier初始化方法初始化第一个神经网络层，
然后将第三个神经网络层初始化为常量值42。

In [12]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.4538, -0.3270, -0.1785, -0.3643])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


### [**自定义初始化**]

有时，深度学习框架没有提供我们需要的初始化方法。
在下面的例子中，我们使用以下的分布为任意权重参数$w$定义初始化方法：

$$
\begin{aligned}
    w \sim \begin{cases}
        U(5, 10) & \text{ 可能性 } \frac{1}{4} \\
            0    & \text{ 可能性 } \frac{1}{2} \\
        U(-10, -5) & \text{ 可能性 } \frac{1}{4}
    \end{cases}
\end{aligned}
$$

同样，我们实现了一个`my_init`函数来应用到`net`。

In [13]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                       for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-0.0000, 0.0000, 7.9345, -0.0000],
        [8.6410, 0.0000, 0.0000, -0.0000]], grad_fn=<SliceBackward0>)

注意，我们始终可以直接设置参数。

[解释一下上面的初始化权重](https://kimi.moonshot.cn/share/cql78augi3pjqbnn94kg)

[补充](https://kimi.moonshot.cn/share/cql798u65ra1tij7fc00)

In [14]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  1.0000,  8.9345,  1.0000])

## [**参数绑定**]

有时我们希望在多个层间共享参数：
我们可以定义一个稠密层，然后使用它的参数来设置另一个层的参数。

In [15]:
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                   shared, nn.ReLU(),
                   shared, nn.ReLU(),
                   nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


这个例子表明第三个和第五个神经网络层的参数是绑定的。
它们不仅值相等，而且由相同的张量表示。
因此，如果我们改变其中一个参数，另一个参数也会改变。
这里有一个问题：当参数绑定时，梯度会发生什么情况？
答案是由于模型参数包含梯度，因此在反向传播期间第二个隐藏层（即第三个神经网络层）和第三个隐藏层（即第五个神经网络层）的梯度会加在一起。

## 小结

* 我们有几种方法可以访问、初始化和绑定模型参数。
* 我们可以使用自定义初始化方法。

## 练习

1. 使用[层和块](https://github.com/lixinjie97/Deep_learning_tutorial/blob/main/04.deep-learning-computation/02.model-construction.ipynb)中定义的`NestMLP`模型，访问各个层的参数。
1. 查看初始化模块文档以了解不同的初始化方法。
1. 构建包含共享参数层的多层感知机并对其进行训练。在训练过程中，观察模型各层的参数和梯度。
1. 为什么共享参数是个好主意？

### 练习一

1. 使用[层和块](https://github.com/lixinjie97/Deep_learning_tutorial/blob/main/04.deep-learning-computation/02.model-construction.ipynb)中定义的`NestMLP`模型，访问各个层的参数。

&emsp;&emsp;引用上节中的`NestMLP`模型，可以使用以下代码访问该模型各个层的参数，输出结果将显示每个层对应的参数名称、形状和具体参数。

In [16]:
import torch
import torch.nn as nn

class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                nn.Linear(64, 20), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

model = NestMLP()

# 访问net层的参数
print('访问net层的参数')
for name, param in model.net.named_parameters():
    print(f"参数名称：{name}, 形状：{param.shape}") 

# 访问linear层的参数
print('\n访问linear层的参数')
for name, param in model.linear.named_parameters():
    print(f"参数名称：{name}, 形状：{param.shape}") 

访问net层的参数
参数名称：0.weight, 形状：torch.Size([64, 20])
参数名称：0.bias, 形状：torch.Size([64])
参数名称：2.weight, 形状：torch.Size([20, 64])
参数名称：2.bias, 形状：torch.Size([20])

访问linear层的参数
参数名称：weight, 形状：torch.Size([16, 32])
参数名称：bias, 形状：torch.Size([16])


### 练习二

2. 查看初始化模块文档以了解不同的初始化方法。

&emsp;&emsp;通过查看深度学习框架文档，有以下初始化方法（参考链接：https://pytorch.org/docs/stable/nn.init.html）
- `torch.nn.init.uniform_(tensor, a=0.0, b=1.0)`：以均匀分布$U(a, b)$初始化输入张量。

- `torch.nn.init.normal_(tensor, mean=0.0, std=1.0)`：以正态分布$N(mean, std^2)$初始化输入张量。

- `torch.nn.init.constant_(tensor, val)`：以一确定数值初始化输入张量。

- `torch.nn.init.ones_(tensor)`：用标量值 1 初始化输入张量。

- `torch.nn.init.zeros_(tensor)`：用标量值 0 初始化输入张量。

- `torch.nn.init.eye_(tensor)`：用单位矩阵初始化二维输入张量。

- `torch.nn.init.xavier_uniform_(tensor, gain=1.0)`：从均匀分布$U(-a, a)$中采样，初始化输入张量，其中$a$的值由如下公式确定

  $$a= gain * \sqrt{\frac{6}{fan_{in}+fan_{out}}}$$

  其中$gain$的取值如下表所示
<style> table { margin: auto;} </style>
非线性函数 | gain值
:----:|:----:
Linear/Identity | 1
Conv1D | 1
Conv2D | 1
Conv3D | 1
Sigmoid | 1
Tanh | $\displaystyle\frac{5}{3}$
ReLU | $\sqrt{2}$
Leaky ReLU | $$\sqrt{\frac{2}{1+negative\_slope^2}}$$
SELU | 1 (adaptive)

- `torch.nn.init.xavier_normal_(tensor, gain=1.0)`:从正态分布$N(0,std^2)$中采样，初始化输入张量，其中$std$值由下式确定：

  $$std= gain * \sqrt{\frac{2}{fan_{in}+fan_{out}}}$$

- `torch.nn.init.kaiming_uniform_(tensor, a=0, mode='fan_in', nonlinearity='leaky_relu')`:服从均匀分布$U(−bound, bound)$，其中$bound$值由下式确定

  $$bound= gain * \sqrt{\frac{3}{fan_{mode}}}$$

- `torch.nn.init.kaiming_normal_(tensor, a=0, mode='fan_in', nonlinearity='leaky_relu')`:服从从正态分布$N(0,std^2)$中采样，其中$std$值由下式确定

  $$std= \frac{gain}{\sqrt{fan_{mode}}}$$
  
- `torch.nn.init.trunc_normal_(tensor, mean=0.0, std=1.0, a=- 2.0, b=2.0)`:用从截断的正态分布中提取的值初始化输入张量。这些值实际上是从正态分布 $N(mean, std^2)$中提取的。

- `torch.nn.init.sparse_(tensor, sparsity, std=0.01)`：将 2D 输入张量初始化为稀疏矩阵，其中非零元素服从正态分布$N(0,0.01)$。  

### 练习三

3. 构建包含共享参数层的多层感知机并对其进行训练。在训练过程中，观察模型各层的参数和梯度。

&emsp;&emsp;在训练过程中，我们每个epoch都打印了每层的参数和梯度。可以看到shared_fc层的参数和梯度都是相同的，因为它们共享同一个参数。

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim

# 模型参数
input_size = 3
hidden_size = 4
output_size = 3
lr = 0.01
epochs = 2

# 构建带有共享参数层的多层感知机
shared_fc = nn.Linear(hidden_size, hidden_size)
MLP = nn.Sequential(nn.Linear(input_size, hidden_size), nn.ReLU(),
                   shared_fc, nn.ReLU(),
                   shared_fc, nn.ReLU(),
                   nn.Linear(hidden_size, output_size))

# 训练数据
X = torch.randn(1, input_size)
Y = torch.randn(1, output_size)
# 优化器
optimizer = optim.SGD(MLP.parameters(), lr=lr)
# 训练模型
for epoch in range(epochs):
    # 前向传播和计算损失
    Y_pred = MLP(X)
    loss = nn.functional.mse_loss(Y_pred, Y)
    # 反向传播和更新梯度
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # 打印每层的参数和梯度
    print(f'Epoch: {epoch}')
    for name, param in MLP.named_parameters():
        print(f"参数名称：{name}, 具体参数值：{param.data[:2]}, 参数梯度：{param.grad[:2]}") # 为了节省页面空间，这里只打印了每个参数和梯度的前两维
    print(f'Epoch：{epoch}, Loss：{loss.item()}\n')

Epoch: 0
参数名称：0.weight, 具体参数值：tensor([[ 0.5548, -0.3248,  0.2388],
        [ 0.1103,  0.0878,  0.4410]]), 参数梯度：tensor([[-0., -0., -0.],
        [-0., -0., -0.]])
参数名称：0.bias, 具体参数值：tensor([-0.0365, -0.4335]), 参数梯度：tensor([0., 0.])
参数名称：2.weight, 具体参数值：tensor([[ 0.4332,  0.2209,  0.3702,  0.3684],
        [ 0.2237, -0.4731,  0.3305, -0.1959]]), 参数梯度：tensor([[ 0.1024,  0.6928,  0.0891,  1.2577],
        [-0.0705, -0.4772,  0.9290,  0.0946]])
参数名称：2.bias, 具体参数值：tensor([-0.3402,  0.3970]), 参数梯度：tensor([1.5441, 0.7866])
参数名称：6.weight, 具体参数值：tensor([[-0.1396, -0.3433,  0.0059, -0.2224],
        [ 0.2504,  0.2871,  0.2662, -0.0786]]), 参数梯度：tensor([[-1.7733e-01, -3.9085e-02, -4.6856e-01, -1.0457e+00],
        [ 2.0208e-03,  4.4541e-04,  5.3397e-03,  1.1917e-02]])
参数名称：6.bias, 具体参数值：tensor([-0.0118,  0.3932]), 参数梯度：tensor([-1.2364,  0.0141])
Epoch：0, Loss：7.444265842437744

Epoch: 1
参数名称：0.weight, 具体参数值：tensor([[ 0.5548, -0.3248,  0.2388],
        [ 0.1103,  0.0878,  0.4410]]), 参数梯度：tensor([[-0

可以看出这个过程只有两个共享层的参数和梯度的数值是一样的，我们直接对这两层结果进行进一步的确认。

In [18]:
# 检查参数是否相同
print(f"参数是否相同：{MLP[2].weight.data == MLP[4].weight.data}")
# 确保它们实际上是同一个对象，而不是有相同的值
MLP[2].weight.data[0, 0] = 100
print(f"参数是否同时变化：{MLP[2].weight.data[0] == MLP[4].weight.data[0]}")
print(f"是否是同一个对象：{MLP[2].weight.data.equal(MLP[4].weight.data)}")

# 检查参数是否相同
print(f"梯度是否相同：{MLP[2].weight.grad == MLP[4].weight.grad}")
# 确保它们实际上是同一个对象，而不是有相同的值
MLP[2].weight.grad[0, 0] = 100
print(f"梯度是否同时变化：{MLP[2].weight.grad[0] == MLP[4].weight.grad[0]}")
print(f"是否是同一个对象：{MLP[2].weight.grad.equal(MLP[4].weight.grad)}")

参数是否相同：tensor([[True, True, True, True],
        [True, True, True, True],
        [True, True, True, True],
        [True, True, True, True]])
参数是否同时变化：tensor([True, True, True, True])
是否是同一个对象：True
梯度是否相同：tensor([[True, True, True, True],
        [True, True, True, True],
        [True, True, True, True],
        [True, True, True, True]])
梯度是否同时变化：tensor([True, True, True, True])
是否是同一个对象：True


### 练习四

4. 为什么共享参数是个好主意？

&emsp;&emsp;1. 节约内存：共享参数可以减少模型中需要存储的参数数量，从而减少内存使用。

&emsp;&emsp;2. 加速收敛：共享参数可以让模型更加稳定，加速收敛。

&emsp;&emsp;3. 提高泛化能力：共享参数可以帮助模型更好地捕捉数据中的共性，提高模型的泛化能力。

&emsp;&emsp;4. 加强模型的可解释性：共享参数可以让模型更加简洁明了，加强模型的可解释性。